In [1]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
from nilearn.image import load_img, resample_to_img, index_img
from nilearn.masking import intersect_masks, apply_mask, unmask
from nilearn.signal import clean
from nilearn.interfaces.fmriprep import load_confounds
import glob
import re
import matplotlib.pyplot as plt


In [5]:
# -----------------------------------------------------------------------------
# 1) USER PARAMETERS
# -----------------------------------------------------------------------------

# Where your fmriprep lives
base_dir    = '/Volumes/Passport/fmriprep'
task_label  = 'pieman'
output_mask = os.path.join(base_dir,
                           f'group_task-{task_label}_intersect_mask.nii.gz')

# TR indices to keep
tr_slice = slice(13, 173)
TR = 1.5

# Exclusions
exclude_subs     = {'sub-001','sub-021','sub-022','sub-038','sub-056','sub-068','sub-069'}
exclude_sub_runs = {('sub-002','2'),('sub-003','2'),('sub-004','2'),
                    ('sub-005','2'),('sub-006','2'),('sub-008','2'),
                    ('sub-010','2'),('sub-011','2'),('sub-012','2'),
                    ('sub-013','2'),('sub-014','2'),('sub-015','2'),
                    ('sub-016','2')}

# Which BOLD space / resolution to use
preferred_space = 'MNI152NLin2009cAsym'
preferred_res   = 'native'

target_subject = None


In [6]:
# -----------------------------------------------------------------------------
# 2) BUILD LIST OF SUBJECTS
# -----------------------------------------------------------------------------
all_subs = sorted(d for d in os.listdir(base_dir)
                  if d.startswith('sub-')
                  and os.path.isdir(os.path.join(base_dir, d)))
if target_subject:
    if target_subject not in all_subs:
        raise ValueError(f"{target_subject} not found under {base_dir}")
    subjects = [target_subject]
else:
    subjects = [s for s in all_subs if s not in exclude_subs]

In [ ]:
# -----------------------------------------------------------------------------
# 3) MAKE GROUP INTERSECT MASK
# -----------------------------------------------------------------------------
# where to save the mask
deriv_mask_dir = os.path.join(base_dir, 'derivatives', 'pieMan_masks')
os.makedirs(deriv_mask_dir, exist_ok=True)
# redefine output_mask to live under derivatives/
output_mask = os.path.join(
    deriv_mask_dir,
    f'group_task-{task_label}_intersect_mask.nii.gz'
)

mask_files = []
space_res  = f"space-{preferred_space}_res-{preferred_res}"
for sub in subjects:
    func_dir = os.path.join(base_dir, sub, 'func')
    # try run-level then single-run masks
    patterns = [
        os.path.join(func_dir,
                     f"{sub}_task-{task_label}_run-1_{space_res}_desc-brain_mask.nii.gz"),
        os.path.join(func_dir,
                     f"{sub}_task-{task_label}_{space_res}_desc-brain_mask.nii.gz"),
    ]
    files = sorted(glob.glob(patterns[0])) or sorted(glob.glob(patterns[1]))
    # make sure files are found
    
    # if not files:
    #     print(f"Skipping: {sub} no mask found")
    
    # get file we want (ie., select run-1, not run-2)
    for fp in files:
        m   = re.search(r"_run-(\d+)_", fp)
        run = m.group(1) if m else None
        if run and (sub,run) in exclude_sub_runs:
            print(f"  • skipping {sub} run-{run}")
            continue
        mask_files.append(fp)

if not mask_files:
    raise RuntimeError("No masks found after exclusions!")

print(f"Found {len(mask_files)} masks, intersecting…")
intersect = intersect_masks(mask_files,
                           threshold=1.0,    
                           connected=False)
intersect.to_filename(output_mask)

mask_img = load_img(output_mask)

print("Saved group intersect mask to:", output_mask)

Skipping sub-083: no mask found
Skipping sub-084: no mask found
Skipping sub-085: no mask found
Skipping sub-086: no mask found
Skipping sub-087: no mask found
Skipping sub-088: no mask found
Skipping sub-089: no mask found
Skipping sub-090: no mask found
Skipping sub-091: no mask found
Skipping sub-092: no mask found
Skipping sub-093: no mask found
Skipping sub-094: no mask found
Skipping sub-095: no mask found
Skipping sub-096: no mask found
Skipping sub-097: no mask found
Skipping sub-098: no mask found
Skipping sub-099: no mask found
Skipping sub-100: no mask found
Skipping sub-101: no mask found
Skipping sub-102: no mask found
Skipping sub-103: no mask found
Skipping sub-104: no mask found
Skipping sub-105: no mask found
Skipping sub-106: no mask found
Skipping sub-107: no mask found
Skipping sub-108: no mask found
Skipping sub-109: no mask found
Skipping sub-110: no mask found
Skipping sub-111: no mask found
Skipping sub-112: no mask found
Skipping sub-113: no mask found
Skipping

In [ ]:
assert len(mask_files) == 75, f"Expected 75 masks, found {len(mask_files)}"

# for future stims (not Pieman) - 
# add a dictaionry at the start to get num subs

In [10]:
len(mask_files)

75